In [1]:
import tkinter as tk
from tkinter import ttk
from tkinter import messagebox
from tkinter import filedialog
import os
from pathlib import Path
import shutil


## Im age Copier Application

This Python script defines a class `ImageCopier` which creates a graphical user interface (GUI) to copy specific types of medical imaging files from a source directory to a destination directory. The application is built using the Tkinter library and provides the following functionalities:

1. **Select Base Folder:**
   - Allows the user to select a base folder that contains subdirectories with medical imaging files.

2. **Display Folder Structure:**
   - Displays the structure of the selected folder in a text area, highlighting subdirectories that match the expected patterns (e.g., `func_rest`, `anat_mprage_anonymized`, `anat_mprage_skullstripped`).

3. **Copy Specific Types of Images:**
   - Provides buttons to copy different types of images (`func_rest`, `anonymized`, `skullstripped`) from the selected base folder to a user-specified destination folder.
   - Copies files with extensions `.nii` and `.nii.gz`.

### Key Components:

- **User Interface:**
  - A main window with buttons to select the base folder and copy specific types of images.
  - A text area that displays logs and the structure of the selected folder.
  
- **Image Types and Patterns:**
  - `func_rest`: Looks for files matching the pattern `scan_rest*.nii.gz` in the `func_rest/NIFTI` subdirectory.
  - `anonymized`: Looks for files matching the pattern `scan_mprage_anonymized*.nii.gz` in the `anat_mprage_anonymized/NIFTI` subdirectory.
  - `skullstripped`: Looks for files matching the pattern `scan_mprage_skullstripped*.nii.gz` in the `anat_mprage_skullstripped/NIFTI` subdirectory.

### Workflow:

1. **Initialization:**
   - The `ImageCopier` class is instantiated, creating a Tkinter window with the necessary UI elements.
   
2. **Folder Selection:**
   - The user selects the base folder containing the image subdirectories using the "Select Base Folder" button.
   
3. **Display Structure:**
   - The folder structure is recursively displayed in the text area, highlighting relevant subdirectories and files.
   
4. **Copying Images:**
   - The user chooses the type of images to copy (`func_rest`, `anonymized`, or `skullstripped`) and selects the destination folder.
   - The application copies the images from the source to the destination folder, updating the text area with logs of the operation.

### Example Usage:

To use this application, run the script. A window will appear allowing you to select the base folder and specify the types of images to copy. Logs will be displayed in the text area, showing the progress and results of the copying process.



In [2]:
class ImageCopier:
    def __init__(self):
        self.root = tk.Tk()
        self.root.title("Copiador de Imágenes AnnArbor")
        self.root.geometry("800x500")
        
        # Variables para almacenar rutas
        self.source_path = None
        
        # Frame principal
        self.main_frame = tk.Frame(self.root)
        self.main_frame.pack(fill=tk.BOTH, expand=True, padx=10, pady=10)
        
        # Frame para botones
        self.button_frame = tk.Frame(self.main_frame)
        self.button_frame.pack(pady=10)
        
        # Botón para seleccionar carpeta
        self.select_button = tk.Button(
            self.button_frame, 
            text="Seleccionar Carpeta Base",
            command=self.select_folder,
            width=20
        )
        self.select_button.pack(side=tk.LEFT, padx=5)
        
        # Frame para botones de copia
        self.copy_buttons_frame = tk.Frame(self.main_frame)
        self.copy_buttons_frame.pack(pady=5)
        
        # Botones para copiar diferentes tipos de imágenes
        self.copy_rest_button = tk.Button(
            self.copy_buttons_frame,
            text="Copiar func_rest",
            command=lambda: self.copy_images('func_rest'),
            width=20,
            state=tk.DISABLED
        )
        self.copy_rest_button.pack(side=tk.LEFT, padx=5)
        
        self.copy_anon_button = tk.Button(
            self.copy_buttons_frame,
            text="Copiar Anonymized",
            command=lambda: self.copy_images('anonymized'),
            width=20,
            state=tk.DISABLED
        )
        self.copy_anon_button.pack(side=tk.LEFT, padx=5)
        
        self.copy_skull_button = tk.Button(
            self.copy_buttons_frame,
            text="Copiar Skullstripped",
            command=lambda: self.copy_images('skullstripped'),
            width=20,
            state=tk.DISABLED
        )
        self.copy_skull_button.pack(side=tk.LEFT, padx=5)
        
        # Text area para mostrar la estructura y logs
        self.log_area = tk.Text(
            self.main_frame,
            wrap=tk.WORD,
            width=80,
            height=20
        )
        self.log_area.pack(pady=10, fill=tk.BOTH, expand=True)
        
        # Scrollbar para el text area
        self.scrollbar = tk.Scrollbar(self.log_area)
        self.scrollbar.pack(side=tk.RIGHT, fill=tk.Y)
        self.log_area.config(yscrollcommand=self.scrollbar.set)
        self.scrollbar.config(command=self.log_area.yview)
    
    def find_nifti_folder(self, path):
        """Busca la carpeta NIFTI independientemente de mayúsculas/minúsculas"""
        if not path.exists():
            return None
        
        for item in path.iterdir():
            if item.is_dir() and item.name.lower() == 'nifti':
                return item
        return None
    
    def select_folder(self):
        folder_path = filedialog.askdirectory()
        if folder_path:
            self.source_path = Path(folder_path)
            self.log_area.delete(1.0, tk.END)
            self.log_area.insert(tk.END, f"Carpeta seleccionada: {folder_path}\n")
            self.show_folder_structure(folder_path)
            self.copy_rest_button.config(state=tk.NORMAL)
            self.copy_anon_button.config(state=tk.NORMAL)
            self.copy_skull_button.config(state=tk.NORMAL)
    
    def show_folder_structure(self, folder_path, level=0):
        self._recursive_list(Path(folder_path), level)
    
    def _recursive_list(self, path, level=0):
        indent = "  " * level
        if "AnnArbor_sub" in path.name:
            self.log_area.insert(tk.END, f"{indent}📁 {path.name}\n")
            
            # Listar archivos en func_rest/NIFTI
            func_rest_path = path / 'func_rest'
            func_nifti = self.find_nifti_folder(func_rest_path)
            if func_nifti:
                self.log_area.insert(tk.END, f"{indent}  📁 func_rest/NIFTI\n")
                self._list_nifti_files(func_nifti, level + 2)
            
            # Listar archivos en anat_mprage_anonymized/NIFTI
            anon_path = path / 'anat_mprage_anonymized'
            anon_nifti = self.find_nifti_folder(anon_path)
            if anon_nifti:
                self.log_area.insert(tk.END, f"{indent}  📁 anat_mprage_anonymized/NIFTI\n")
                self._list_nifti_files(anon_nifti, level + 2)
            
            # Listar archivos en anat_mprage_skullstripped/NIFTI
            skull_path = path / 'anat_mprage_skullstripped'
            skull_nifti = self.find_nifti_folder(skull_path)
            if skull_nifti:
                self.log_area.insert(tk.END, f"{indent}  📁 anat_mprage_skullstripped/NIFTI\n")
                self._list_nifti_files(skull_nifti, level + 2)
        else:
            for item in sorted(path.iterdir()):
                if item.is_dir():
                    self._recursive_list(item, level)
    
    def _list_nifti_files(self, path, level):
        indent = "  " * level
        try:
            for item in sorted(path.iterdir()):
                if item.is_file() and (item.suffix in ['.gz', '.nii'] or item.name.endswith('.nii.gz')):
                    self.log_area.insert(tk.END, f"{indent}📄 {item.name}\n")
        except PermissionError:
            self.log_area.insert(tk.END, f"{indent}⚠️ Sin acceso\n")
    
    def copy_images(self, image_type):
        if not self.source_path:
            return
        
        # Mapeo de tipos de imagen a rutas y patrones de archivo
        type_config = {
            'func_rest': {
                'path': 'func_rest',
                'pattern': 'scan_rest*.nii.gz'
            },
            'anonymized': {
                'path': 'anat_mprage_anonymized',
                'pattern': 'scan_mprage_anonymized*.nii.gz'
            },
            'skullstripped': {
                'path': 'anat_mprage_skullstripped',
                'pattern': 'scan_mprage_skullstripped*.nii.gz'
            }
        }
        
        # Seleccionar carpeta destino
        dest_folder = filedialog.askdirectory(title=f"Seleccionar carpeta destino para {image_type}")
        if not dest_folder:
            return
        
        dest_path = Path(dest_folder)
        self.log_area.delete(1.0, tk.END)
        self.log_area.insert(tk.END, f"Iniciando copia de imágenes {image_type}...\n")
        
        # Contador para las imágenes copiadas
        copied_count = 0
        errors_count = 0
        
        config = type_config[image_type]
        
        # Buscar todas las carpetas AnnArbor_sub
        for ann_folder in self.source_path.glob('**/AnnArbor_sub*'):
            if not ann_folder.is_dir():
                continue
            
            # Obtener el ID del sujeto del nombre de la carpeta
            subject_id = ann_folder.name
            
            # Buscar la carpeta NIFTI correspondiente
            img_path = ann_folder / config['path']
            nifti_path = self.find_nifti_folder(img_path)
            
            if not nifti_path:
                self.log_area.insert(tk.END, f"⚠️ No se encontró {config['path']}/NIFTI en {subject_id}\n")
                errors_count += 1
                continue
            
            # Copiar archivos .nii y .nii.gz
            for img_file in nifti_path.glob('*'):
                if img_file.suffix in ['.gz', '.nii'] or img_file.name.endswith('.nii.gz'):
                    # Crear nuevo nombre de archivo con el ID del sujeto
                    new_filename = f"{subject_id}_{img_file.name}"
                    dest_file = dest_path / new_filename
                    
                    try:
                        shutil.copy2(img_file, dest_file)
                        self.log_area.insert(tk.END, f"✅ Copiado: {new_filename}\n")
                        copied_count += 1
                    except Exception as e:
                        self.log_area.insert(tk.END, f"❌ Error al copiar {new_filename}: {str(e)}\n")
                        errors_count += 1
                    
                    # Actualizar la interfaz
                    self.root.update()
        
        self.log_area.insert(tk.END, f"\nProceso completado:\n")
        self.log_area.insert(tk.END, f"- Archivos copiados exitosamente: {copied_count}\n")
        self.log_area.insert(tk.END, f"- Errores encontrados: {errors_count}")
    
    def run(self):
        self.root.mainloop()

if __name__ == "__main__":
    app = ImageCopier()
    app.run()